In [1]:
import pandas as pd

In [14]:
df = pd.read_csv('AviationData.csv', header=0, encoding='cp1252',dtype=str) #'cp1252' encoding becuase of weird ascii values

In [17]:
df.columns = df.columns.str.replace('.', ' ') # Change '.' in column names to ' '

In [40]:
pd.set_option('display.max_columns', 500) # keeps getting cut off
df.head(40)

,Event Id,Investigation Type,Accident Number,Event Date,Location,Country,Latitude,Longitude,Airport Code,Airport Name,Injury Severity,Aircraft damage,Aircraft Category,Registration Number,Make,Model,Amateur Built,Number of Engines,Engine Type,FAR Description,Schedule,Purpose of flight,Air carrier,Total Fatal Injuries,Total Serious Injuries,Total Minor Injuries,Total Uninjured,Weather Condition,Broad phase of flight,Report Status,Publication Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,NC6404,Stinson,108-3,No,1,Reciprocating,NaN,NaN,Personal,NaN,2,0,0,0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,Fatal(4),Destroyed,NaN,N5069P,Piper,PA24-180,No,1,Reciprocating,NaN,NaN,Personal,NaN,4,0,0,0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,Fatal(3),Destroyed,NaN,N5142R,Cessna,172M,No,1,Reciprocating,NaN,NaN,Personal,NaN,3,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,Fatal(2),Destroyed,NaN,N1168J,Rockwell,112,No,1,Reciprocating,NaN,NaN,Personal,NaN,2,0,0,0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,Fatal(1),Destroyed,NaN,N15NY,Cessna,501,No,NaN,NaN,NaN,NaN,Personal,NaN,1,2,NaN,0,VMC,Approach,Probable Cause,16-04-1980
5,20170710X52551,Accident,NYC79AA106,1979-09-17,"BOSTON, MA",United States,42.445277,-70.758333,NaN,NaN,Non-Fatal,Substantial,Airplane,CF-TLU,Mcdonnell Douglas,DC9,No,2,Turbo Fan,Part 129: Foreign,SCHD,NaN,Air Canada,NaN,NaN,1,44,VMC,Climb,Probable Cause,19-09-2017
6,20001218X45446,Accident,CHI81LA106,1981-08-01,"COTTON, MN",United States,NaN,NaN,NaN,NaN,Fatal(4),Destroyed,NaN,N4988E,Cessna,180,No,1,Reciprocating,NaN,NaN,Personal,NaN,4,0,0,0,IMC,Unknown,Probable Cause,06-11-2001
7,20020909X01562,Accident,SEA82DA022,1982-01-01,"PULLMAN, WA",United States,NaN,NaN,NaN,BLACKBURN AG STRIP,Non-Fatal,Substantial,Airplane,N2482N,Cessna,140,No,1,Reciprocating,Part 91: General Aviation,NaN,Personal,NaN,0,0,0,2,VMC,Takeoff,Probable Cause,01-01-1982
8,20020909X01561,Accident,NYC82DA015,1982-01-01,"EAST HANOVER, NJ",United States,NaN,NaN,N58,HANOVER,Non-Fatal,Substantial,Airplane,N7967Q,Cessna,401B,No,2,Reciprocating,Part 91: General Aviation,NaN,Business,NaN,0,0,0,2,IMC,Landing,Probable Cause,01-01-1982
9,20020909X01560,Accident,MIA82DA029,1982-01-01,"JACKSONVILLE, FL",United States,NaN,NaN,JAX,JACKSONVILLE INTL,Non-Fatal,Substantial,NaN,N3906K,North American,NAVION L-17B,No,1,Reciprocating,NaN,NaN,Personal,NaN,0,0,3,0,IMC,Cruise,Probable Cause,01-01-1982


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Event Id                88889 non-null  object
 1   Investigation Type      88889 non-null  object
 2   Accident Number         88889 non-null  object
 3   Event Date              88889 non-null  object
 4   Location                88837 non-null  object
 5   Country                 88663 non-null  object
 6   Latitude                34382 non-null  object
 7   Longitude               34373 non-null  object
 8   Airport Code            50132 non-null  object
 9   Airport Name            52704 non-null  object
 10  Injury Severity         87889 non-null  object
 11  Aircraft damage         85695 non-null  object
 12  Aircraft Category       32287 non-null  object
 13  Registration Number     87507 non-null  object
 14  Make                    88826 non-null  object
 15  Mo

In [34]:
df.isnull().sum().sort_values(ascending = False) # Ranked missing data

Schedule                  76307
Air carrier               72241
FAR Description           56866
Aircraft Category         56602
Longitude                 54516
Latitude                  54507
Airport Code              38757
Airport Name              36185
Broad phase of flight     27165
Publication Date          13771
Total Serious Injuries    12510
Total Minor Injuries      11933
Total Fatal Injuries      11401
Engine Type                7096
Report Status              6384
Purpose of flight          6192
Number of Engines          6084
Total Uninjured            5912
Weather Condition          4492
Aircraft damage            3194
Registration Number        1382
Injury Severity            1000
Country                     226
Amateur Built               102
Model                        92
Make                         63
Location                     52
Investigation Type            0
Event Date                    0
Accident Number               0
Event Id                      0
dtype: i

In [41]:
"""
Things we don't care about:

    Schedule
    FAR Description
    Aircraft Category:
        Helicopters
    Longitude
    Latitude
    Airport Code
    Airport Name
    Location
    Broad phase of flight
    **Publication Date*** maybe important, newer planes might be better
    **Report status** not sure what it means
    Weather condition 
    Air Carrier
    Registration number
    Country
    Location
    Investigation type
    **Event date** again, maybe important
    Accident number
    Event ID

    
Things we do care about:

    Total fatal injuries
    Total minor injuries
    Total serious injuries
    Aircraft Damage
    Engine Type
    Injury severity
    **Ameteur built** remove any data tied to this
    Model
    Make
    
"""

"\nThings we don't care about:\n\n    Schedule\n    FAR Description\n    Aircraft Category:\n        Helicopters\n    Longitude\n    Latitude\n    Airport Code\n    Airport Name\n    Location\n    Broad phase of flight\n    **Publication Date*** maybe important, newer planes might be better\n    **Report status** not sure what it means\n    Weather condition \n    Air Carrier\n    Registration number\n    Country\n    Location\n    Investigation type\n    **Event date** again, maybe important\n    Accident number\n    Event ID\n\n    \nThings we do care about:\n\n    Total fatal injuries\n    Total minor injuries\n    Total serious injuries\n    Aircraft Damage\n    Engine Type\n    Injury severity\n    **Ameteur built** remove any data tied to this\n    Model\n    Make\n    \n"